In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from causalinference.causal import CausalModel
from causaldata import black_politicians

In [2]:
br = black_politicians.load_pandas().data
br.head(20)

,leg_black,treat_out,responded,totalpop,medianhhincom,black_medianhh,white_medianhh,blackpercent,statessquireindex,nonblacknonwhite,urbanpercent,leg_senator,leg_democrat,south
0,0,0,0,1.5873,5.0625,2.6814,2.6586,0.007119,0.227,0,0.695601,0,0,0
1,0,0,1,1.6218,4.9713,2.7126,2.6619,0.005796,0.227,0,0.618073,0,0,0
2,0,0,1,1.6710,6.9646,2.3087,2.9973,0.012029,0.227,0,0.824331,0,0,0
3,0,0,1,1.6122,4.1811,2.4668,2.4887,0.004280,0.227,1,0.000000,0,0,0
4,0,1,1,1.5622,3.1152,2.1490,2.0597,0.008258,0.227,1,0.000000,0,1,0
5,0,1,0,1.6236,6.0678,1.9856,2.7017,0.011826,0.227,0,0.358323,0,0,0
6,0,1,1,1.6426,5.5030,2.4090,2.7584,0.027030,0.227,0,0.382215,0,1,0
7,0,0,0,1.6872,4.2315,1.7252,2.4060,0.097736,0.227,1,0.997706,0,1,0
8,0,0,1,3.2091,5.0087,2.6918,2.6602,0.006450,0.227,0,0.657326,1,0,0
9,0,1,0,3.1744,3.6754,2.2020,2.3071,0.006237,0.227,1,0.000000,1,1,0


In [3]:
# Converting pandas DataFrame columns to NumPy arrays for further processing

# Change the names of two features to make them more readable
br.rename(columns={
    'leg_black': 'black_legislator', 
    'leg_democrat': 'dem_legislator', 
    'medianhhincom':'median_hh_income',
    'blackpercent': 'black_percent'}, inplace=True)

# Outcome variable - whether a legislator responded or not
responded = br['responded'].to_numpy()

# Treatment - indicator for whether the individual is a black legislator
is_black_legislator = br['black_legislator'].to_numpy()

# Matching variables - median household income, percentage of black population, and indicator for Democrat legislator
matching_variables = br[['median_hh_income', 'black_percent', 'dem_legislator']].to_numpy()

In [4]:
# Setting up the causal model
M = CausalModel(
    responded,
    is_black_legislator,
    matching_variables)

In [5]:
# Estimating the propensity score
M.est_propensity()
# Trim the sample based on the propensity score
M.trim_s()
# Export the weights
br['ps'] = M.propensity['fitted']
# Estimate the effects
M.est_via_weighting()
print(M.estimates)


Treatment Effect Estimates: Weighting

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.047      0.080      0.588      0.557     -0.109      0.203



c:\Users\rodri\Dropbox\github\CausalInference\.venv\Lib\site-packages\causalinference\estimators\weighting.py:23: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  wlscoef = np.linalg.lstsq(Z_w, Y_w)[0]


In [6]:
# Create our own weights
br['ipw'] = np.where(br['black_legislator'] == 1, 1 / br['ps'], 1 / (1 - br['ps']))
# Estimate the effects
m = smf.wls('responded ~ black_legislator', br, weights=br['ipw']).fit()
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:              responded   R-squared:                       0.022
Model:                            WLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     128.6
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.75e-29
Time:                        00:00:54   Log-Likelihood:                -5768.6
No. Observations:                5593   AIC:                         1.154e+04
Df Residuals:                    5591   BIC:                         1.155e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.4100      0.009     43.596      0.000       0.392       0.428
black_legislator     0.1499      0.013     11.340      0.000       0.124       0.176
==============================================================================
Omnibus:                     2184.371   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2062976.257
Skew:                           0.078   Prob(JB):                         0.00
Kurtosis:                      97.087   Cond. No.                         2.63
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""